In [73]:
import pandas as pd


In [ ]:
from gene_tools import foldersLoad, cleanDic, compute_scores, geneScores, NA_filtering
from Scoring.scomet import Mean, Max, Min, Median, pca

root_path = r"C:\Users\alber\Downloads\00_BPStart\00_BPStart\00_gene_prioritization"
folders = ["GWAS", "eQTL_GWAS_blood", "pQTL_GWAS", "Exome"]
raw_data = foldersLoad(root_path, folders)
dic_clean = cleanDic(raw_data)
merged = pd.read_csv(r"C:\Users\alber\Downloads\00_BPStart\00_BPStart\merged_drug_all_db_with_hgnc_TTD.tsv", sep="\t")

In [112]:
from Scoring.scomet import Mean, Max, Min, Median, pca

ldl_clean = dic_clean["LDL"]

Max(ldl_clean)
Mean(ldl_clean)
Median(ldl_clean)
Min(ldl_clean)

score_cols = ["Prioscore_mean", "Prioscore_max", "Prioscore_min", "Prioscore_median"]
drug_targets = set(merged[merged['trait'] == 'LDL'].loc[merged[merged['trait'] == 'LDL']["Sum"] >= 3,"EnsemblId"])   
                                                                                                             
for col in score_cols:
    top = ldl_clean.loc[ldl_clean[col] < 0.5]
    top_ids = set(top["EnsemblId"])
    overlap = top_ids & drug_targets
    percent = 100 * len(overlap) / len(top_ids)
    print(f"{col}: {percent:.2f}% of top 1% are drug targets")
    
for col in score_cols:
    top = ldl_clean.sort_values(by=col, ascending=True).head(int(0.01 * len(ldl_clean)))
    top_ids = set(top["EnsemblId"])
    overlap = top_ids & drug_targets
    percent = 100 * len(overlap) / len(top_ids)
    print(f"{col}: {percent:.2f}% of top 1% are drug targets")
len(ldl_clean["GWAS_percentile"].dropna())

Prioscore_mean: 20.00% of top 1% are drug targets
Prioscore_max: 7.69% of top 1% are drug targets
Prioscore_min: 6.01% of top 1% are drug targets
Prioscore_median: 22.22% of top 1% are drug targets
Prioscore_mean: 4.41% of top 1% are drug targets
Prioscore_max: 2.94% of top 1% are drug targets
Prioscore_min: 6.37% of top 1% are drug targets
Prioscore_median: 4.90% of top 1% are drug targets


1929

In [97]:


ldl_clean["LDL"] = NA_filtering(ldl_clean["LDL"])
ldl_clean ["LDL"] = pca(ldl_clean["LDL"])
Max(ldl_clean["LDL"])
Mean(ldl_clean["LDL"])
Median(ldl_clean["LDL"])
Min(ldl_clean["LDL"])
print(len(ldl_clean["LDL"]["Prioscore_min"]))

from gene_tools.analysis import evaluate_prioritization

evaluate_prioritization(df,ldl_clean["LDL"],sum_threshold=2,top_percent = 0.01,score_cols = ["Prioscore_mean", "Prioscore_max", "Prioscore_min", "Prioscore_median","PCA"])


1387
Prioscore_mean: OR = 0.57, p = 7.4492e-01, drug targets in top 1.0% = 2
Prioscore_max: OR = 0.26, p = 2.0838e-01, drug targets in top 1.0% = 1
Prioscore_min: OR = 0.94, p = 1.0000e+00, drug targets in top 1.0% = 3
Prioscore_median: OR = 0.94, p = 1.0000e+00, drug targets in top 1.0% = 3
PCA: OR = 0.57, p = 7.4492e-01, drug targets in top 1.0% = 2
